In [13]:
import csv
import pandas as pd
import numpy as np
import os

In [14]:
DF_GENDER_DIR = "../csv-files/NewAUData_gender.csv"
DF_DURR_DIR = "csv-files/NewAUData.csv" 
CSV_DIR = "csv-files"
NEW_CSV_DIR = "../csv-files/AudioClips.csv"
AUDIO_CLIPS_DIR = "../dataset/AudioClips"

In [15]:
df_gender = pd.read_csv(DF_GENDER_DIR)
df_gender.shape

(220, 6)

In [16]:
df_gender.columns

Index(['Name', 'audioID', 'Folder', 'Class', 'Duration', 'Gender'], dtype='object')

In [17]:
df_gender = df_gender[['audioID', 'Class', 'Gender']]
df_gender.head()

,audioID,Class,Gender
0,Audio_01_00001.mp3,1,M
1,Audio_01_00002.mp3,1,M
2,Audio_01_00003.mp3,1,M
3,Audio_01_00004.mp3,1,M
4,Audio_01_00005.mp3,1,M


In [18]:
df_gender['audioID'] = df_gender['audioID'].apply(lambda x: x[:-4])
df_gender.head()

,audioID,Class,Gender
0,Audio_01_00001,1,M
1,Audio_01_00002,1,M
2,Audio_01_00003,1,M
3,Audio_01_00004,1,M
4,Audio_01_00005,1,M


In [19]:
def save_csv_files(data, csvfile):
    with open(csvfile, "w") as csv_file:
        writer = csv.writer(csv_file)
        writer.writerows(data)
    csv_file.close()

def append_csv_files(data, csvfile):
    with open(csvfile, "a") as csv_file:
        writer = csv.writer(csv_file)
        writer.writerows(data)
    csv_file.close()

In [20]:
audioClip_csv = [['audioID', 'Class','Time_IN', 'Time_OUT', 'Gender']]

In [21]:
list_au_clips = os.listdir(AUDIO_CLIPS_DIR)
list_au_clips.sort()

In [84]:
test_audio_file = 'Audio_01_00221_2730s_to_2760s.mp3'

au_clip_id = test_audio_file[:14]
print(au_clip_id)

au_clip_class = 1 # default 1

au_clip_time = test_audio_file[16:-3].split("_")
# print(au_clip_time)
print(au_clip_time[0][:-1], au_clip_time[-1][:-2])
au_clip_time_in, au_clip_time_out = int(au_clip_time[0][:-1]), int(au_clip_time[-1][:-2])

au_clip_gender = df_gender[df_gender['audioID'] == au_clip_id]['Gender'].values[0]
print(type(au_clip_gender), au_clip_gender)



Audio_01_00221
730 2760
<class 'str'> F


In [86]:
len(list_au_clips)

8875

In [91]:
save_csv_files(audioClip_csv,NEW_CSV_DIR)

In [92]:
df_au_clips = pd.read_csv(NEW_CSV_DIR, index_col=False)
df_au_clips.shape

(8875, 5)

In [96]:
8875 / 2, 

(4437.5,)

In [93]:
df_au_clips.head()

,audioID,Class,Time_IN,Time_OUT,Gender
0,Audio_01_00001,1,0,30,M
1,Audio_01_00001,1,1020,1050,M
2,Audio_01_00001,1,1050,1080,M
3,Audio_01_00001,1,1080,1110,M
4,Audio_01_00001,1,1110,1140,M


In [95]:
df_au_clips['Gender'].value_counts()

Gender
M    4493
F    4382
Name: count, dtype: int64

In [ ]:
4382 * 100 / 8875 # apporx 50%

49.374647887323945

# Gender Classification using VGR AI

In [ ]:
# TODO:
# API testing  of VGR AI
# gender function testing 
# 

In [3]:
import os

os.getcwd()

'/Users/eyash.p24/Desktop/MUJ/ML/ATD_ML/research'

In [4]:
API_KEY_DIR = "../API/vgr_api_key.json"

In [8]:
import json

api_key = json.load(open(API_KEY_DIR))


In [9]:
WAV_FILE_DIR = "../dataset/WAVClips"

In [35]:
from pydub import AudioSegment

In [36]:
def mp3_wav_converter(mp3_file, wav_file_dir):
    os.makedirs(wav_file_dir, exist_ok=True)

    sound = AudioSegment.from_mp3(mp3_file)
    mp3_file = os.path.split(mp3_file)[-1]
    wav_file = os.path.join(wav_file_dir, mp3_file[:-4] + ".wav")
    sound.export(wav_file, format="wav")
    
    return wav_file

In [38]:
au_test_file = list_au_clips[1]
au_file_path = os.path.join(AUDIO_CLIPS_DIR, au_test_file)

wavfile = mp3_wav_converter(au_file_path, WAV_FILE_DIR)
wavfile

'../dataset/WAVClips/Audio_01_00001_1020s_to_1050s.wav'

In [29]:
import requests

def gender(file_path,file_name,api_key):
    # API endpoint
    api_url = "https://api.genderrecognition.com/v1/voice-gender-recognition/api"

    # API endpoint
    api_url = "https://api.genderrecognition.com/v1/voice-gender-recognition/api"

    mp3_wav_converter(file_path,WAV_FILE_DIR)
    wav_file = file_path[:-4] + '.wav'

    # Open the file and send the request
    try:
        with open(wav_file, 'rb') as file:
            files = {"file": (file_name, file, "audio/wav")}
            headers = {"apiKey": api_key['API_KEY']}
            response = requests.post(api_url, files=files, headers=headers)

        # Check response status
        if response.status_code == 200:
            print("API Response:", response.json())
        else:
            print(f"Request failed with status code {response.status_code}: {response.text}")

    except FileNotFoundError:
        print(f"File not found: {file_path}")
    except requests.RequestException as e:
        print(f"Error making request: {e}")
    
    return response.json()


In [30]:
au_test_file = list_au_clips[1]
au_file_path = os.path.join(AUDIO_CLIPS_DIR, au_test_file)
# print(au_file_path)

gen = gender(au_file_path, au_test_file, api_key)

FileNotFoundError: [Errno 2] No such file or directory: '../dataset/WAVClips/Audio_01_00001_1020s_to_1050s.wav'

In [ ]:
# audioClip_csv = [['audioID', 'Class','Time_IN', 'Time_OUT' 'Gender']]

for au_clip in list_au_clips:
    print(au_clip)
    au_clip_id = au_clip[:14]
    au_clip_class = 1
    au_clip_time = au_clip[15:-3].split("_")
    print(au_clip_time)
    au_clip_time_in, au_clip_time_out = int(au_clip_time[0][:-1]), int(au_clip_time[-1][:-2])
    au_clip_gender =  df_gender[df_gender['audioID'] == au_clip_id]['Gender'].values[0]

    if au_clip_gender == "F":
        pass
        

    audioClip_csv.append([au_clip_id, au_clip_class, au_clip_time_in, au_clip_time_out, au_clip_gender])
    